In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from datetime import datetime, date
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
FILES_DIR = './files/'
total = pd.read_csv(FILES_DIR + 'total.csv')

In [3]:
group_data = pd.DataFrame({'count': total.groupby(['session_id', 'user_no', 'item_no', 'gender', 'platform']).size()})
group_data = group_data.reset_index()

group_data2 = total.groupby('session_id')['rating'].mean().reset_index()

group_data3 = pd.merge(group_data, group_data2, on='session_id', how='inner')

group_data3 = group_data3

group_data = group_data3.sort_values(by='rating', ascending=False)

# group_data = group_data.drop_duplicates(subset='session_id', keep='first')

group_data = group_data[group_data['count'] == 1]

group_data

,session_id,user_no,item_no,gender,platform,count,rating
0,00047f91f7624d57ae28662b751624db,hoEZFZeX2JUz9+RmAgvubQ==,iP6hnNonEYCWvtV/wsjilA==,F,ANDROID,1,4.0
12100,bba908b4a15aec7237d3f81e93af4eec,X2EtZNswNUi0NQ811Lk0ZA==,Gx7FQgogekHlIMM3ELkHtQ==,F,IOS,1,4.0
15410,ed936abf233af281ff9a6a93152fd9c6,1PZIaHU/KKbfd+DlVyj0RA==,0whP9oGO0f1EHkVqP4gxfg==,F,IOS,1,4.0
1498,179804d87499ad125855bda4a5e9309e,Q7Mc9PJSqgCI5ozZOsc3HA==,6nccKk24PQkrfPhw7cTsOQ==,M,IOS,1,4.0
12139,bc354cf7b5f838d2b1e8a51a0282acdd,/A0eX40c7U/3uiLG/akkTA==,18N28EpFbjcDiUeef3TPug==,F,ANDROID,1,4.0
...,...,...,...,...,...,...,...
6126,5fbfa5744a5b91041a6eeb4ecc15111b,laoJc6XwqswFhQ+6BB0p2A==,MWgv/9pRTxstSyMM2dF9dA==,F,IOS,1,1.0
6127,5fc33d9111e1105614bd8edbdf3f18b2,ccQ8/m6yuvjZKiWD+/KHww==,KGADz14c9pHc+BuwReOQBw==,F,IOS,1,1.0
6128,5fc39b28f229c9b49342e5befce306c9,j1y/B/jOUQd0mb2vyQIvBQ==,y8glepMabn1ukjk43CJWCg==,F,IOS,1,1.0
6129,5fc4f901e508009d863fc365fac5f061,n6dD9L7YfVfC+vxvIk1eMQ==,up5cIpRinQ618fkewQzbWg==,F,IOS,1,1.0


In [4]:
# train, test split
x = group_data.copy()
y = group_data['session_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


def RMSE(y_true, y_pred):
  return np.sqrt(np.mean(np.power(np.array(y_true)-np.array(y_pred), 2)))


def score(model, neighbor_size=0): # neighbor_size 파라미터 추가
  id_pairs = zip(x_test['session_id'], x_test['item_no'])
  y_pred = np.array([model(session_id, item_no, neighbor_size) for (session_id, item_no) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

In [5]:
rating_matrix = group_data.pivot_table(index='session_id', columns='item_no', values='rating') 
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = pd.DataFrame(cosine_similarity(matrix_dummy), index=rating_matrix.index, columns=rating_matrix.index)

In [6]:
# 각 사용자의 평점평균 구하기
rating_mean = rating_matrix.mean(axis=1)
# 각 아이템의 평점을 각 사용자의 평균에서의 차이로 변환하기
rating_bias = (rating_matrix.T - rating_mean).T


rating_binary1 = np.array((rating_matrix>0).astype(float))
rating_binary2 = rating_binary1.T
counts = pd.DataFrame(np.dot(rating_binary1, rating_binary2), 
                      index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

In [7]:
def CF_knn_bias_sig_adj(session_id, item_no, neighbor_size=0):
  if item_no in rating_bias:
    sim_scores = user_similarity[session_id].copy()
    item_ratings = rating_bias[item_no].copy()
    
    no_rating = item_ratings.isnull() # 해당 아이템에 대해 rating 하지 않은 사용자를 True로 표시
    common_counts = counts[session_id]
    low_significance = common_counts < SIG_LEVEL # 유저간 공통 평가 아이템수가가 SIG_LEVEL보다 낮은 사용자를 True 표시 
    none_rating_idx = item_ratings[no_rating | low_significance].index # no_rating이 True이거나 low_significance가 True인 사용자 표시
    
    item_ratings = item_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    
    if neighbor_size == 0:
      prediction = np.dot(sim_scores, item_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[session_id]

    else:
      if len(sim_scores) > MIN_RATINGS: # 해당 아이템 rating 유저수가 MIN_RATINGS보다 큰 경우 예측값 계산
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        item_ratings = np.array(item_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        item_ratings = item_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, item_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[session_id]
      else:
        prediction = rating_mean[session_id]

  else:
    prediction = rating_mean[session_id]
  
  # 이 부분 추가
  if prediction < 1.0:	
    prediction = 1.0
  elif prediction > 4.0:
    prediction = 4.0
    
  return prediction
  
SIG_LEVEL = 3
MIN_RATINGS = 2
print('최종 예측 값 수정한 협업 필터링 방식의 정확도:', score(CF_knn_bias_sig_adj, 30))

최종 예측 값 수정한 협업 필터링 방식의 정확도: 0.0
